# Shape Detection

La Transformada de Hough es una técnica que permite detectar geometrías en una imagen, mientras esta pueda ser parametrizada matemáticamente. Mediante su matriz acumuladora de votos, logra detectar recta y círculos incluso si estos presentan ruido o irregularidades.

En este worshop utilizaremos las implementaciones de la Transformada de Hough en OpenCV para trackear y orientar un cuaderno en una imagen.

<img src="https://raw.githubusercontent.com/cherrerab/roboticafcfm/master/auxiliar_02/bin/banner.png" height="200">

## OpenCV

La Open Surce Computer Vision es una librería especializada en herramientas de visión computacional y en todo lo que respecta a visión artificial en general. De este modo provee de funcionalidades de todo tipo de complejidad, desde operaciones básicas de procesamiento de imágenes, hasta algoritmos de reconocimiento de objetos.

Por supuesto, esta librería cuenta con su documentación correspondiente. En esta puede encontrar descripciones más detalladas de sus funcionalidades, así como también ejemplos, tutoriales y otros.

https://docs.opencv.org/master/

<img src="https://raw.githubusercontent.com/cherrerab/roboticafcfm/master/auxiliar_01/bin/opencv_logo.png" height="200">

## Load Image
La imagen `ref_01.jpg` con que trabajaremos en este workshop se encuentra en el repositorio del curso, de este modo lo primero que haremos será clonar este github.

In [ ]:
!git clone https://github.com/cherrerab/roboticafcfm.git
%cd /content/roboticafcfm

Ahora, antes de utilizar la Transformadad de Hough debemos procesar la imagen tal de aislar lo más posible la geometría o forma que queremos detectar. Para esto, dependiendo de las características de la imagen, bien podemos usar binarización directa o segementación por color.

Primero analicemos el contenido de la imagen `ref_01.jpg`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import cv2

# dirección de la imagen
img_path = 'auxiliar_02//ref_01.jpg'

# cargar imagen
img = 

# transformar a RGB
img_rgb = 

# print shape de img
print('\nimg.shape:', img.shape)

# plotear imagen en RGB
plt.figure( figsize=(10,10) )
plt.imshow( img_rgb )

De esta manera, podemos ver que `ref.jpg` consiste en la foto de un cuaderno verde inclinado. Aprovecharemos este color para utilizar segementación por color y así asilar el cuaderno.

## Color Segmentation

Utilizaremos la misma estrategia de segmentacióón que en la auxiliar anterior. Analizaremos la distribución de valores `HSV` en la imagen y a partir de esta determinaremos un rango para binarizar la imagen y generar el `mask`.

In [ ]:
# transformar imagen a HSV
img_hsv = 

# extraer canal HUE de la imagen
img_hue = 

# plotear canal HUE de la imagen
plt.figure( figsize=(9, 9) )
plt.title( 'Hue Map' )
plt.imshow( img_hue, cmap='hsv')

# extraer canal SAT de la imagen
img_sat = 

# plotear canal SAT de la imagen
plt.figure( figsize=(9, 9) )
plt.title( 'Saturation Map' )
plt.imshow( img_sat, cmap='jet')

# extraer valor de un punto del cuaderno
sample = img_hsv[300, 600, :]
print('valores HSV de referencia: ', sample)

Podemos observar que en los distintos canales HSV los valores son sumamente uniformes en lo que respecta a la geometría del cuaderno. Tomando como valores de referencia `(70 221 127)`, definiremos nuestro rango de segmentación en torno a este punto.

In [ ]:
# definir límite inferior
lower_hsv = 

# definir límite superior
upper_hsv = 

# segmentar mediante cv2.inRange
mask = cv2.inRange(img_hsv, lower_hsv, upper_hsv)

# visualizar resultado
plt.figure( figsize=(9, 9) )
plt.title( 'Mask' )

plt.imshow(mask, cmap='gray')

Hemos logrado aislar el cuaderno completamente del resto de la imagen. No obstante, dado que la Transformada de Hough de rectas requiere, por supuesto, que existan rectas en la imagen, tenemos que extraer los bordes de esta imagen para poder procesarla. Para esto podemos utilizar el método `cv2.Canny`.

[Documentación Canny Edge Detection OpenCV](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_canny/py_canny.html)

In [ ]:
# extraer bordes mediante cv2.Canny
edges = 

# visualizar bordes resultantes
plt.figure( figsize=(9, 9) )
plt.title( 'Edges' )

plt.imshow(edges, cmap='gray')

Perfecto! Ahora podemos aplicar la Transformada de Hough para detectar y trackear las rectas en la imagen.

## Hough Line Transform

Todo el algoritmo de la Transformada de Hough ha sido implementado en OpenCV mediante el método `cv2.HoughLines`. Este método retorna un `np.array` con los parámetros `(rho, theta)` de cada una de las rectas detectadas. Por supuesto, para operar requiere una imagen binaria como nuestra imagen `edges`, la resolución de búsqueda para los parámetros `(rho, theta)`, y el umbral de votos para segmentar las rectas detectadas.

[Documentación Hough Lines OpenCV](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_houghlines/py_houghlines.html)

In [ ]:
# extraer rectas mediante cv2.HoughLines
rho_res, theta_res = 1, 2*np.pi/180
threshold =

lines = cv2.HoughLines(edges, rho_res, theta_res, threshold)

# print resultado de rectas
print('rectas detectadas:\n', lines)

Notemos que el resultado consiste en una lista de listas. Por lo que al procesar esta información debemos cuidar el como la extraemos.

Ahora bien, una vez tenemos los parámetros `(rho, theta)` para cada recta, podemos transformar esta parametrización tal de poder plotear las rectas identificadas en la imagen mediante la función `cv2.line`.

In [ ]:
# cargar imagen
ret = cv2.imread('auxiliar_02//ref_01.jpg')
ret = cv2.cvtColor(ret, cv2.COLOR_BGR2RGB)

# para cada recta detectada
for line in lines:
  # obtener parámetros (rho, theta)
  rho, theta = line[0]

  a = np.cos(theta)
  b = np.sin(theta)
  x0 = a*rho
  y0 = b*rho

  # obtener recta (x1,y1), (x2,y2)
  x1 = int(x0 + 1000*(-b))
  y1 = int(y0 + 1000*(a))
  x2 = int(x0 - 1000*(-b))
  y2 = int(y0 - 1000*(a))

  # plotear recta en la imagen con cv2.line
  color = (0, 255, 255)
  thickness = 4
  ret = cv2.line(ret, (x1, y1), (x2, y2), color, thickness)

# visualizar rectas resultantes
plt.figure( figsize=(9, 9) )
plt.title( 'Hough Lines' )

plt.imshow(ret, cmap='gray')